In [1]:
%%writefile daemon.json
{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    },
    "data-root": "/home/ec2-user/SageMaker/docker"
}

Writing daemon.json


In [2]:
%%bash
sudo systemctl stop docker
sudo cp daemon.json /etc/docker/daemon.json
sudo cp -rp /var/lib/docker /home/ec2-user/SageMaker/docker
sudo systemctl start docker

  docker.socket


In [43]:
!docker build --quiet --tag api-layer:latest .

sha256:e50c8555c34747667c12700e368a64d2f127f0631592b98849df773df285b50c


In [35]:
!docker run --name test-api-layer -d -p 8001:8001 -e AWS_REGION=us-west-2 -e AWS_DEFAULT_REGION=us-west-2 api-layer:latest --host 0.0.0.0 --port 8001 --table-name api-layer-ddb

7ac490de66723e02a06bf48a15ac1c64ea3df8fb5b386722ad2fbd00c0e1c854


In [3]:
!docker context ls

NAME        DESCRIPTION                               DOCKER ENDPOINT               KUBERNETES ENDPOINT   ORCHESTRATOR
default *   Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                         swarm


In [48]:
host_ip = "internal-api-layer-internal-1190385235.us-west-2.elb.amazonaws.com"#"0.0.0.0"

In [49]:
%%bash -s $host_ip
curl ${1}:8001/list_models

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1156  100  1156    0     0  21222      0 --:--:-- --:--:-- --:--:-- 21407


{"models":[{"model_type":"Claude","model_name":"anthropic.claude-instant-v1","model_family":"bedrock","streaming":true},{"model_type":"Claude","model_name":"anthropic.claude-v1","model_family":"bedrock","streaming":true},{"model_type":"Claude","model_name":"anthropic.claude-v2","model_family":"bedrock","streaming":true},{"model_type":"Titan","model_name":"amazon.titan-tg1-large","model_family":"bedrock","streaming":true},{"model_type":"Jurassic","model_name":"ai21.j2-grande-instruct","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-jumbo-instruct","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-mid","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-mid-v1","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-ultra-v1","model_family":"bedrock","streaming":false},{"model_type":"Cohere","model_name":"cohere.command-text-v14","m

In [51]:
%%bash -s $host_ip
curl -X POST ${1}:8001/invoke -d '{"body":{"prompt": "Human: Hello, write a poem about New Zealand\nAssistant:"}, "model_family": "bedrock", "model_name": "ai21.j2-ultra-v1"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   824  100   684  100   140    260     53  0:00:02  0:00:02 --:--:--   313


{"generated_text":"\nNew Zealand! The new Zeeland!\nA place for new beginnings,\nA chance to start over,\nIn a new land, with new opportunities,\nA place to escape to,\nA place to rediscover yourself,\nA place to call your own.\n\nIt's a place where nature reigns,\nWhere you can walk in the wilderness,\nAnd listen as the waves crash against the shore,\nWhere the sun shines brightly on a clear day,\nAnd the moon illuminates the night sky,\nWhere the air is clean and crisp,\nAnd the water is pure and refreshing.\n\nNew Zealand! The new Zeeland!\nA place where dreams can come true,\nA place to discover your destiny,\nA place to call home.","finish_reason":{"reason":"endoftext"}}

In [52]:
%%bash -s $host_ip
curl -X POST ${1}:8001/invoke_stream -d '{"body":{"prompt": "Human: Hello, write a poem about New Zealand\nAssistant:"}, "model_family": "bedrock", "model_name": "anthropic.claude-v2"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1788    0  1645  100   143    161     14  0:00:10  0:00:10 --:--:--   104


{"generated_text": " Here", "finish_reason": null}
{"generated_text": " is a poem about New Zealand:\n\nOde to", "finish_reason": null}
{"generated_text": " New Zealand\n\nLand of mountains, fjords and plains\nIsl", "finish_reason": null}
{"generated_text": "ands scattered, coasts with", "finish_reason": null}
{"generated_text": " crashing waves\nForests thick with", "finish_reason": null}
{"generated_text": " native trees\nBirdsong drifting", "finish_reason": null}
{"generated_text": " on the breeze\n\nSheep dot rolling", "finish_reason": null}
{"generated_text": " emerald hills \nVine", "finish_reason": null}
{"generated_text": "yards stretch across the land\nGlaciers carve", "finish_reason": null}
{"generated_text": " valleys, rivers flow\nMisty", "finish_reason": null}
{"generated_text": " rainforests down below\n\nMaori culture, history alive", "finish_reason": null}
{"generated_text": "\nKiwi spirit,", "finish_reason": null}
{"generated_text": " adventure thrives \nRugby", "finis

In [37]:
!docker logs test-api-layer

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
api-layer-ddb
INFO:     172.17.0.1:52678 - "GET /list_models HTTP/1.1" 200 OK


In [44]:
!docker kill test-api-layer && docker rm test-api-layer

Error response from daemon: Cannot kill container: test-api-layer: No such container: test-api-layer


In [45]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 705484421587.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [46]:
!docker tag api-layer:latest 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer:latest

In [47]:
!docker push 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer:latest

The push refers to repository [705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer]

d2e01d21: Preparing 
e5b7ac0c: Preparing 
f0b21a82: Preparing 
f9eb716b: Preparing 
ffbeec5d: Preparing 
7097618f: Preparing 
ab009408: Preparing 
ec988e2a: Preparing 
fd493130: Preparing 
e5b7ac0c: Pushed   87.36MB/85.2MBBlatest: digest: sha256:0d1db56fb488267413b4e39a6802b7c8d103f80501814b88b75886f22a5c0223 size: 2416


In [41]:
!docker images

REPOSITORY                                               TAG         IMAGE ID       CREATED        SIZE
api-layer                                                latest      45b20bbe7f99   22 hours ago   237MB
705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer   latest      45b20bbe7f99   22 hours ago   237MB
python                                                   3.10-slim   b477fabe1167   2 weeks ago    128MB


In [42]:
! docker rmi -f 45

Untagged: 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer:latest
Untagged: 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer@sha256:41882f490df0621237c4c85434924c324ebc6df13ba8e6e3a6fed0ab6d5cfd09
Untagged: api-layer:latest
Deleted: sha256:45b20bbe7f997130799f6f89962025ac000eeee184c7ddc51908d024d25cbf3f
Deleted: sha256:db8f29a137c3b79f0db7385918acecedf7744b80f9ed5ff90cb43feaabaf7e86
Deleted: sha256:cae76ac263ee1a70bea244bb08846371b801ca3ef786690a307ce060d806aaca
Deleted: sha256:eb737ce9e358434f548da99019981b4a27d15998c59d5e69f908effe2bf7bc71
Deleted: sha256:0af1d311ada067060e13ebff330fe478012a481b2a5696c891ca31c1b25d7f33
Deleted: sha256:e738e6d4e3c5f7efe71f2d24bd6d32458936fc328fa2684176abf0e8d6844c78
Deleted: sha256:45f6b73122b7b865810d8a17c5ab4ab005c56a1384509ee0302e37b4dadb1318
Deleted: sha256:8ff175b6d237424c023bb9117a7dcfb988fb623794e6e16a4d0875fe691b5bae
Deleted: sha256:5dac540975de9ae8a90f81b14e0aae8faf3ec5a2c4c645d020e2620ee76d6f46
Deleted: sha256:7b43a65cfb51739bfe3